## Get Dataset ##

In [1]:
import glob
from collections import defaultdict
from collections import Counter
import ast
import itertools

In [24]:
fr = open("./lang_games_data.txt", "w")
list_files = glob.glob("./cubeturk/examples/*.lisp")
data = []
pl_id = defaultdict(int)
for file in list_files:
    with open(file,'r') as f:
        context = ""
        sess_id = ""
        target = ""
        utterance = ""
        for line in f:
            if "id session" in line:
                start = 'id session:'
                end = ')'
                sess_id = line[line.find(start)+len(start):line.rfind(end)]
                pl_id[sess_id] += 1
            elif "context (" in line:
                start = 'graph NaiveKnowledgeGraph ((string '
                end = ') (name b)'
                context = line[line.find(start)+len(start):line.rfind(end)]
            elif "(utterance " in line:
                start = '(utterance'
                end = ')'
                utterance = line[line.find(start)+len(start):line.rfind(end)][1:].replace('"','')
            elif "(targetValue" in line:
                start = '(targetValue (string '
                end = '))'
                target = line[line.find(start)+len(start):line.rfind(end)]
            elif line == ")\n":
                fr.write(sess_id + "\t" + context + "\t" + utterance + "\t" + target + "\n")
                context = ""
                sess_id = "",
                target = ""
                utterance = ""
fr.close()

In [10]:
#fr = open("./lang_games_data.txt", "w")
list_files = glob.glob("./dataset/sida wang's/lisps/*.lisp")
data = []
pl_id = defaultdict(int)
count = 0
for file in list_files:
    start = "./dataset/sida wang's/lisps/"
    end = '.lisp'
    file_name = file[file.find(start)+len(start):file.rfind(end)]
    fw = open("./dataset/sida wang's/txt/" + file_name + ".txt", "w")
    with open(file,'r') as f:
        context_list = []
        target_list = []
        utterance = ""
        skip = False
        #fw = open("./dataset/sida wang's/*.lisp")
        for line in f:
            if "context (" in line:
                count += 1
                start = 'graph NaiveKnowledgeGraph ((string '
                end = ') (name b)'
                context = line[line.find(start)+len(start):line.rfind(end)]
                counter = Counter(context)
                try:
                    context_list = ast.literal_eval(context)
                except:
                    skip = True
                    continue
                len_prev = len(context_list)
                for el in context_list:
                    if len(el) > 3:
                        skip = True
                if len_prev >= 7:
                    skip = True
#                 else:
#                 if len_prev < 7:
#                     res_len = 7 - len_prev - 1
#                     for _ in range(res_len):
#                         context_list.append(["X"])
                for i, el in enumerate(context_list):
                    rs = 3 - len(el)
                    for j in range(rs):
                        context_list[i].append("X")
                context_list = list(itertools.chain(*context_list))
                for i in range(1,len_prev):
                    context_list.insert(i*3+i-1,'#')
            elif "(utterance " in line and not skip:
                start = '(utterance'
                end = ')'
                utterance = line[line.find(start)+len(start):line.rfind(end)][1:].replace('"','')
            elif "(targetValue" in line and not skip:
                start = '(targetValue (string '
                end = '))'
                context = line[line.find(start)+len(start):line.rfind(end)]
                counter = Counter(context)
                target_list = ast.literal_eval(context)
                len_prev = len(target_list) 
                
                for el in target_list:
                    if len(el) > 3:
                        skip = True
                if len_prev >= 7:
                    skip = True
                
#                 if len_prev < 7:
#                     res_len = 7 - len_prev - 1
#                     for _ in range(res_len):
#                         target_list.append(["X"])
                        
                for i, el in enumerate(target_list):
                    rs = 3 - len(el)
                    for j in range(rs):
                        target_list[i].append("X")
                len_prev = len(target_list)
                target_list = list(itertools.chain(*target_list))
                for i in range(1,len_prev):
                    target_list.insert(i*3+i-1,'#')
            elif line == ")\n":
                if not skip:
                    fw.write(" ".join(str(bc) for bc in context_list)
                        + "\t" + utterance + "\t" + " ".join(str(bc) for bc in target_list) + "\n")
                    #print(" ".join(str(bc) for bc in context_list)
                    #    + "\t" + utterance + "\t" + " ".join(str(bc) for bc in target_list))
                context_list = []
                target_list = []
                utterance = ""
                skip = False
    fw.close()
#             if "id session" in line:
#                 start = 'id session:'
#                 end = ')'
#                 sess_id = line[line.find(start)+len(start):line.rfind(end)]
#                 pl_id[sess_id] += 1
#             elif "context (" in line:
#                 start = 'graph NaiveKnowledgeGraph ((string '
#                 end = ') (name b)'
#                 context = line[line.find(start)+len(start):line.rfind(end)]
#             elif "(utterance " in line:
#                 start = '(utterance'
#                 end = ')'
#                 utterance = line[line.find(start)+len(start):line.rfind(end)][1:].replace('"','')
#             elif "(targetValue" in line:
#                 start = '(targetValue (string '
#                 end = '))'
#                 target = line[line.find(start)+len(start):line.rfind(end)]
#             elif line == ")\n":
#                 fr.write(sess_id + "\t" + context + "\t" + utterance + "\t" + target + "\n")
#                 context = ""
#                 sess_id = "",
#                 target = ""
#                 utterance = ""
# fr.close()
print(count)

10131


## Generate Dataset ##

In [1]:
from random import randint
import random
import copy
import itertools

In [2]:
# we assume that there are 6 tiles and maximum 3 blocks
color_blocks = ["0", "1", "2", "3"] # cyan, brown, red, orange
color_strings = ["cyan", "brown", "red", "orange"]
operation = ["remove", "add"]
positional_info = ["1st", "2nd", "3rd", "4th", "5th", "6th", "even", "odd", "leftmost", "rightmost", "every"]
num_tiles, max_block, num_color = 6, 3, 3

In [14]:
def generate_train_data(num_tiles, max_block, num_color, opr, added_set):
    match = False
    ## user command ##
    #opr = random.choice(operation)
    clr = random.choice(color_strings)
    pos = random.choice(positional_info)
    if opr == "remove": 
        info = " at "
    else:
        info = " to "
    command = opr + " " +  clr + info + pos + " tile"
    #command = "remove cyan at 1st tile"
    
    ## initial state ##
    already_added = True
    while already_added:
        block_config = []
        for i in range(num_tiles): # loop through all the tiles
            num_blocks = randint(0,max_block)
            if num_blocks == 0:
                block_config.append([])
            else:
                blocks = []
                for j in range(num_blocks):
                    block_color_idx = randint(0,num_color)
                    blocks.append(color_blocks[block_color_idx])
                block_config.append(blocks)
        if str(block_config) not in added_set:
            already_added = False
            
    ## generate next state ##
    next_block_config =  copy.deepcopy(block_config)
    tile_indexs = [-1]
    if "1" in command or "leftmost" in command:
        tile_indexs = [0]
    elif "2" in command:
        tile_indexs = [1]
    elif "3" in command:
        tile_indexs = [2]
    elif "4" in command:
        tile_indexs = [3]
    elif "5" in command:
        tile_indexs = [4]
    elif "6" in command or "rightmost" in command:
        tile_indexs = [5]
    elif "even" in command:
        tile_indexs = [1,3,5]
    elif "odd" in command:
        tile_indexs = [0,2,4]
    elif "every" in command:
        tile_indexs = [0,1,2,3,4,5]
        
    for tile_index in tile_indexs:
        if "remove" in command:
            if len(next_block_config[tile_index]) != 0:
                if "cyan" in command:
                    if next_block_config[tile_index][-1] == "0":
                        next_block_config[tile_index].pop()
                        match = True
                elif "brown" in command:
                    if next_block_config[tile_index][-1] == "1":
                        next_block_config[tile_index].pop()
                        match = True
                elif "red" in command:
                    if next_block_config[tile_index][-1] == "2":
                        next_block_config[tile_index].pop()
                        match = True
                elif "orange" in command:
                    if next_block_config[tile_index][-1] == "3":
                        next_block_config[tile_index].pop()
                        match = True
        elif "add" in command and len(next_block_config[tile_index]) < 3:
            if "cyan" in command:
                next_block_config[tile_index].append("0")
                match = True
            elif "brown" in command:
                next_block_config[tile_index].append("1")
                match = True
            elif "red" in command:
                next_block_config[tile_index].append("2")
                match = True
            elif "orange" in command:
                next_block_config[tile_index].append("3")
                match = True
    if match:
        added_set.add(str(block_config))
        
    # change block_config
    for i, el in enumerate(block_config):
        rs = 3 - len(el)
        for j in range(rs):
            block_config[i].append("X")
    block_config = list(itertools.chain(*block_config))
    for i in range(1,6):
        block_config.insert(i*3+i-1,'#')
    
    for i, el in enumerate(next_block_config):
        rs = 3 - len(el)
        for j in range(rs):
            next_block_config[i].append("X")
    next_block_config = list(itertools.chain(*next_block_config))
    for i in range(1,6):
        next_block_config.insert(i*3+i-1,'#')
            
    return block_config, command, next_block_config, match, added_set

In [4]:
total = 20000

f = open("./lang_games_data_artificial_train_nvl_state_" + str(total) + ".txt", "w")
ft = open("./lang_games_data_artificial_test_nvl_state_" + str(total) + ".txt", "w")
added_set = set()

half = 0.5 * total
train_composition = 0.8 * half
for i in range(total):
    opr = "add"
    if i < half:
        opr = "remove"
    match = False

    while not match:
        block_config, command, next_block_config, match, added_set = generate_train_data(
            num_tiles, max_block, num_color, opr, added_set)
        if match:
            if (i >= train_composition and i <half) or (i >= (train_composition + half) and i < total):
                ft.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
            else:
                f.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
f.close()
ft.close()

## General Command

In [12]:
def generate_train_data_novel(num_tiles, max_block, num_color, opr, blocks_b, utter_b, 
                              added_set, blocks, command_set):
    match = False    
    ## user command ##
    if utter_b:
        command = random.choice(command_set)
    else:
        opr = random.choice(operation)
        clr = random.choice(color_strings)
        pos = random.choice(positional_info)
        if opr == "remove": 
            info = " at "
        else:
            info = " to "
        command = opr + " " +  clr + info + pos + " tile"
    
    ## initial state ##
    already_added = True
    if blocks_b:
        while already_added:
            block_config = []
            block_cfg = random.sample(blocks, num_tiles)
            for i in range(num_tiles): # loop through all the tiles
                x = list(block_cfg[i])
                x = list(filter(lambda a: a != 'X', x))
                block_config.append(x)
            if str(block_config) not in added_set:
                already_added = False
    else:
        while already_added:
            block_config = []
            for i in range(num_tiles): # loop through all the tiles
                num_blocks = randint(0,max_block)
                if num_blocks == 0:
                    block_config.append([])
                else:
                    blocks = []
                    for j in range(num_blocks):
                        block_color_idx = randint(0,num_color)
                        blocks.append(color_blocks[block_color_idx])
                    block_config.append(blocks)
            if str(block_config) not in added_set:
                already_added = False
    ## generate next state ##
    next_block_config =  copy.deepcopy(block_config)
    tile_indexs = [-1]
    if "1" in command or "leftmost" in command:
        tile_indexs = [0]
    elif "2" in command:
        tile_indexs = [1]
    elif "3" in command:
        tile_indexs = [2]
    elif "4" in command:
        tile_indexs = [3]
    elif "5" in command:
        tile_indexs = [4]
    elif "6" in command or "rightmost" in command:
        tile_indexs = [5]
    elif "even" in command:
        tile_indexs = [1,3,5]
    elif "odd" in command:
        tile_indexs = [0,2,4]
    elif "every" in command:
        tile_indexs = [0,1,2,3,4,5]
        
    for tile_index in tile_indexs:
        if "remove" in command:
            if len(next_block_config[tile_index]) != 0:
                if "cyan" in command:
                    if next_block_config[tile_index][-1] == "0":
                        next_block_config[tile_index].pop()
                        match = True
                elif "brown" in command:
                    if next_block_config[tile_index][-1] == "1":
                        next_block_config[tile_index].pop()
                        match = True
                elif "red" in command:
                    if next_block_config[tile_index][-1] == "2":
                        next_block_config[tile_index].pop()
                        match = True
                elif "orange" in command:
                    if next_block_config[tile_index][-1] == "3":
                        next_block_config[tile_index].pop()
                        match = True
        elif "add" in command and len(next_block_config[tile_index]) < 3:
            if "cyan" in command:
                next_block_config[tile_index].append("0")
                match = True
            elif "brown" in command:
                next_block_config[tile_index].append("1")
                match = True
            elif "red" in command:
                next_block_config[tile_index].append("2")
                match = True
            elif "orange" in command:
                next_block_config[tile_index].append("3")
                match = True
    if match:
        added_set.add(str(block_config))
        
    # change block_config
    for i, el in enumerate(block_config):
        rs = 3 - len(el)
        for j in range(rs):
            block_config[i].append("X")
    block_config = list(itertools.chain(*block_config))
    for i in range(1,6):
        block_config.insert(i*3+i-1,'#')
    
    for i, el in enumerate(next_block_config):
        rs = 3 - len(el)
        for j in range(rs):
            next_block_config[i].append("X")
    next_block_config = list(itertools.chain(*next_block_config))
    for i in range(1,6):
        next_block_config.insert(i*3+i-1,'#')
            
    return block_config, command, next_block_config, match, added_set

## Generate Novel Dataset - Blocks

In [10]:
alphabet = ["0","1","2","3","X"]
novel_amount = 15
blocks = [''.join(p) for p in itertools.product(alphabet, repeat=3)]
blocks_n = []
for bl in blocks:
    if 'X' in bl[1] and bl[2] is not 'X':
        blocks_n.append(bl)
    elif 'X' in bl[0] and bl[1] is not 'X':
        blocks_n.append(bl)
# length : 85
blocks_fin = [x for x in blocks if x not in blocks_n]
#blocks = list(itertools.chain(*(itertools.permutations(alphabet, i) for i in range(3,3+1))))
#blocks = list(map(lambda x:"".join(x), blocks))
blocks_test = random.sample(blocks_fin, novel_amount)
blocks_mt = [x for x in blocks_fin if x not in blocks_test]
blocks_valid = random.sample(blocks_mt, novel_amount)
blocks = [x for x in blocks_mt if x not in blocks_valid]

In [6]:
def generate_train_data_novel(num_tiles, max_block, num_color, opr, added_set, blocks):
    match = False
    ## user command ##
    #opr = random.choice(operation)
    clr = random.choice(color_strings)
    pos = random.choice(positional_info)
    if opr == "remove": 
        info = " at "
    else:
        info = " to "
    command = opr + " " +  clr + info + pos + " tile"
    #command = "remove cyan at 1st tile"
    
    ## initial state ##
    already_added = True
    while already_added:
        block_config = []
        block_cfg = random.sample(blocks, num_tiles)
        for i in range(num_tiles): # loop through all the tiles
            x = list(block_cfg[i])
            x = list(filter(lambda a: a != 'X', x))
            block_config.append(x)
        if str(block_config) not in added_set:
            already_added = False
            
    ## generate next state ##
    next_block_config =  copy.deepcopy(block_config)
    tile_indexs = [-1]
    if "1" in command or "leftmost" in command:
        tile_indexs = [0]
    elif "2" in command:
        tile_indexs = [1]
    elif "3" in command:
        tile_indexs = [2]
    elif "4" in command:
        tile_indexs = [3]
    elif "5" in command:
        tile_indexs = [4]
    elif "6" in command or "rightmost" in command:
        tile_indexs = [5]
    elif "even" in command:
        tile_indexs = [1,3,5]
    elif "odd" in command:
        tile_indexs = [0,2,4]
    elif "every" in command:
        tile_indexs = [0,1,2,3,4,5]
        
    for tile_index in tile_indexs:
        if "remove" in command:
            if len(next_block_config[tile_index]) != 0:
                if "cyan" in command:
                    if next_block_config[tile_index][-1] == "0":
                        next_block_config[tile_index].pop()
                        match = True
                elif "brown" in command:
                    if next_block_config[tile_index][-1] == "1":
                        next_block_config[tile_index].pop()
                        match = True
                elif "red" in command:
                    if next_block_config[tile_index][-1] == "2":
                        next_block_config[tile_index].pop()
                        match = True
                elif "orange" in command:
                    if next_block_config[tile_index][-1] == "3":
                        next_block_config[tile_index].pop()
                        match = True
        elif "add" in command and len(next_block_config[tile_index]) < 3:
            if "cyan" in command:
                next_block_config[tile_index].append("0")
                match = True
            elif "brown" in command:
                next_block_config[tile_index].append("1")
                match = True
            elif "red" in command:
                next_block_config[tile_index].append("2")
                match = True
            elif "orange" in command:
                next_block_config[tile_index].append("3")
                match = True
    if match:
        added_set.add(str(block_config))
        
    # change block_config
    for i, el in enumerate(block_config):
        rs = 3 - len(el)
        for j in range(rs):
            block_config[i].append("X")
    block_config = list(itertools.chain(*block_config))
    for i in range(1,6):
        block_config.insert(i*3+i-1,'#')
    
    for i, el in enumerate(next_block_config):
        rs = 3 - len(el)
        for j in range(rs):
            next_block_config[i].append("X")
    next_block_config = list(itertools.chain(*next_block_config))
    for i in range(1,6):
        next_block_config.insert(i*3+i-1,'#')
        
    return block_config, command, next_block_config, match, added_set

In [13]:
total_train = 16000
total_test = 4000
total_valid = 4000
total = total_train + total_test + total_valid

f = open("./lang_games_data_artificial_train_nvl_blocks_" + str(total) + ".txt", "w")
ft = open("./lang_games_data_artificial_test_nvl_blocks_" + str(total) + ".txt", "w")
fv = open("./lang_games_data_artificial_valid_nvl_blocks_" + str(total) + ".txt", "w")
added_set = set()

for i in range(total_train):
    opr = "add"
    if i < 0.5 * total_train:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_train:
            block_config, command, next_block_config, match, added_set = generate_train_data_novel(
                num_tiles, max_block, num_color, opr, True, False, added_set, blocks, set())
            if match:
                f.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_novel(
                num_tiles, max_block, num_color, opr, True, False, added_set, blocks, set())
            if match:
                f.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
f.close()

for i in range(total_test):
    opr = "add"
    if i < 0.5 * total_test:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_test:
            block_config, command, next_block_config, match, added_set = generate_train_data_novel(
                num_tiles, max_block, num_color, opr, True, False, added_set, blocks_test, set())
            if match:
                ft.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_novel(
                num_tiles, max_block, num_color, opr, True, False, added_set, blocks_test, set())
            if match:
                ft.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
ft.close()

for i in range(total_valid):
    opr = "add"
    if i < 0.5 * total_valid:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_test:
            block_config, command, next_block_config, match, added_set = generate_train_data_novel(
                num_tiles, max_block, num_color, opr, True, False, added_set, blocks_valid, set())
            if match:
                fv.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_novel(
                num_tiles, max_block, num_color, opr, True, False, added_set, blocks_valid, set())
            if match:
                fv.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
fv.close()

## Generate Novel Dataset - Utterance

In [4]:
def add_info(string):
    ins = string.split(" ")
    ins.insert(3, "tile")
    if ins[0] == "remove":
        ins.insert(2, "at")
    elif ins[0] == "add":
        ins.insert(2, "to")
    return " ".join(ins)
command_list = [["remove", "add"],["cyan", "brown", "red", "orange"],["1st", "2nd", "3rd", "4th", "5th", "6th", "even", "odd", "leftmost", "rightmost", "every"]]
# length : 88
novel_amount = 10
command = [add_info(" ".join(p)) for p in itertools.product(*command_list)]
r_command = command[:44]
a_command = command[44:]
rmv_test = random.sample(r_command, novel_amount)
add_test = random.sample(a_command, novel_amount)
add_command = [x for x in a_command if x not in add_test]
rmv_command = [x for x in r_command if x not in rmv_test]
#command_fin = [add_info(val) for pair in zip(add_command, rmv_command) for val in pair]
#command_fin_test = [add_info(val) for pair in zip(add_test, rmv_test) for val in pair]

In [114]:
def generate_train_data_utter(num_tiles, max_block, num_color, opr, added_set, command_set):
    match = False
    ## user command ##
    #opr = random.choice(operation)
#     clr = random.choice(color_strings)
#     pos = random.choice(positional_info)
#     if opr == "remove": 
#         info = " at "
#     else:
#         info = " to "
#     command = opr + " " +  clr + info + pos + " tile"
    #command = "remove cyan at 1st tile"
    command = random.choice(command_set)
    
    ## initial state ##
    already_added = True
    while already_added:
        block_config = []
        for i in range(num_tiles): # loop through all the tiles
            num_blocks = randint(0,max_block)
            if num_blocks == 0:
                block_config.append([])
            else:
                blocks = []
                for j in range(num_blocks):
                    block_color_idx = randint(0,num_color)
                    blocks.append(color_blocks[block_color_idx])
                block_config.append(blocks)
        if str(block_config) not in added_set:
            already_added = False
            
    ## generate next state ##
    next_block_config =  copy.deepcopy(block_config)
    tile_indexs = [-1]
    if "1" in command or "leftmost" in command:
        tile_indexs = [0]
    elif "2" in command:
        tile_indexs = [1]
    elif "3" in command:
        tile_indexs = [2]
    elif "4" in command:
        tile_indexs = [3]
    elif "5" in command:
        tile_indexs = [4]
    elif "6" in command or "rightmost" in command:
        tile_indexs = [5]
    elif "even" in command:
        tile_indexs = [1,3,5]
    elif "odd" in command:
        tile_indexs = [0,2,4]
    elif "every" in command:
        tile_indexs = [0,1,2,3,4,5]
        
    for tile_index in tile_indexs:
        if "remove" in command:
            if len(next_block_config[tile_index]) != 0:
                if "cyan" in command:
                    if next_block_config[tile_index][-1] == "0":
                        next_block_config[tile_index].pop()
                        match = True
                elif "brown" in command:
                    if next_block_config[tile_index][-1] == "1":
                        next_block_config[tile_index].pop()
                        match = True
                elif "red" in command:
                    if next_block_config[tile_index][-1] == "2":
                        next_block_config[tile_index].pop()
                        match = True
                elif "orange" in command:
                    if next_block_config[tile_index][-1] == "3":
                        next_block_config[tile_index].pop()
                        match = True
        elif "add" in command and len(next_block_config[tile_index]) < 3:
            if "cyan" in command:
                next_block_config[tile_index].append("0")
                match = True
            elif "brown" in command:
                next_block_config[tile_index].append("1")
                match = True
            elif "red" in command:
                next_block_config[tile_index].append("2")
                match = True
            elif "orange" in command:
                next_block_config[tile_index].append("3")
                match = True
    if match:
        added_set.add(str(block_config))
        
    # change block_config
    for i, el in enumerate(block_config):
        rs = 3 - len(el)
        for j in range(rs):
            block_config[i].append("X")
    block_config = list(itertools.chain(*block_config))
    for i in range(1,6):
        block_config.insert(i*3+i-1,'#')
    
    for i, el in enumerate(next_block_config):
        rs = 3 - len(el)
        for j in range(rs):
            next_block_config[i].append("X")
    next_block_config = list(itertools.chain(*next_block_config))
    for i in range(1,6):
        next_block_config.insert(i*3+i-1,'#')
            
    return block_config, command, next_block_config, match, added_set

In [115]:
total_train = 16000
total_test = 4000
total = total_train + total_test

f = open("./lang_games_data_artificial_train_nvl_utter_" + str(total) + ".txt", "w")
ft = open("./lang_games_data_artificial_test_nvl_utter_" + str(total) + ".txt", "w")
added_set = set()

for i in range(total_train):
    opr = "add"
    if i < 0.5 * total_train:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_train:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, rmv_command)
            if match:
                f.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, add_command)
            if match:
                f.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
f.close()

for i in range(total_test):
    opr = "add"
    if i < 0.5 * total_test:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_test:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, rmv_test)
            if match:
                ft.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, add_test)
            if match:
                ft.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
ft.close()

## Generate Novel Dataset - Blocks and Utterance

In [116]:
def generate_train_data_utter(num_tiles, max_block, num_color, opr, added_set, blocks, command_set):
    match = False
    ## user command ##
    #opr = random.choice(operation)
#     clr = random.choice(color_strings)
#     pos = random.choice(positional_info)
#     if opr == "remove": 
#         info = " at "
#     else:
#         info = " to "
#     command = opr + " " +  clr + info + pos + " tile"
    #command = "remove cyan at 1st tile"
    command = random.choice(command_set)
    
    ## initial state ##
    already_added = True
    while already_added:
        block_config = []
        block_cfg = random.sample(blocks, num_tiles)
        for i in range(num_tiles): # loop through all the tiles
            x = list(block_cfg[i])
            x = list(filter(lambda a: a != 'X', x))
            block_config.append(x)
        if str(block_config) not in added_set:
            already_added = False
            
    ## generate next state ##
    next_block_config =  copy.deepcopy(block_config)
    tile_indexs = [-1]
    if "1" in command or "leftmost" in command:
        tile_indexs = [0]
    elif "2" in command:
        tile_indexs = [1]
    elif "3" in command:
        tile_indexs = [2]
    elif "4" in command:
        tile_indexs = [3]
    elif "5" in command:
        tile_indexs = [4]
    elif "6" in command or "rightmost" in command:
        tile_indexs = [5]
    elif "even" in command:
        tile_indexs = [1,3,5]
    elif "odd" in command:
        tile_indexs = [0,2,4]
    elif "every" in command:
        tile_indexs = [0,1,2,3,4,5]
        
    for tile_index in tile_indexs:
        if "remove" in command:
            if len(next_block_config[tile_index]) != 0:
                if "cyan" in command:
                    if next_block_config[tile_index][-1] == "0":
                        next_block_config[tile_index].pop()
                        match = True
                elif "brown" in command:
                    if next_block_config[tile_index][-1] == "1":
                        next_block_config[tile_index].pop()
                        match = True
                elif "red" in command:
                    if next_block_config[tile_index][-1] == "2":
                        next_block_config[tile_index].pop()
                        match = True
                elif "orange" in command:
                    if next_block_config[tile_index][-1] == "3":
                        next_block_config[tile_index].pop()
                        match = True
        elif "add" in command and len(next_block_config[tile_index]) < 3:
            if "cyan" in command:
                next_block_config[tile_index].append("0")
                match = True
            elif "brown" in command:
                next_block_config[tile_index].append("1")
                match = True
            elif "red" in command:
                next_block_config[tile_index].append("2")
                match = True
            elif "orange" in command:
                next_block_config[tile_index].append("3")
                match = True
    if match:
        added_set.add(str(block_config))
        
    # change block_config
    for i, el in enumerate(block_config):
        rs = 3 - len(el)
        for j in range(rs):
            block_config[i].append("X")
    block_config = list(itertools.chain(*block_config))
    for i in range(1,6):
        block_config.insert(i*3+i-1,'#')
    
    for i, el in enumerate(next_block_config):
        rs = 3 - len(el)
        for j in range(rs):
            next_block_config[i].append("X")
    next_block_config = list(itertools.chain(*next_block_config))
    for i in range(1,6):
        next_block_config.insert(i*3+i-1,'#')
            
    return block_config, command, next_block_config, match, added_set

In [117]:
total_train = 16000
total_test = 4000
total = total_train + total_test

f = open("./lang_games_data_artificial_train_nvl_utter&blocks_" + str(total) + ".txt", "w")
ft = open("./lang_games_data_artificial_test_nvl_utter&blocks_" + str(total) + ".txt", "w")
added_set = set()

for i in range(total_train):
    opr = "add"
    if i < 0.5 * total_train:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_train:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, blocks, rmv_command)
            if match:
                f.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, blocks, add_command)
            if match:
                f.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
f.close()

for i in range(total_test):
    opr = "add"
    if i < 0.5 * total_test:
        opr = "remove"
    match = False

    while not match:
        if i < 0.5 * total_test:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, blocks_test, rmv_test)
            if match:
                ft.write(" ".join(bc for bc in block_config) 
                         + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
        else:
            block_config, command, next_block_config, match, added_set = generate_train_data_utter(
                num_tiles, max_block, num_color, opr, added_set, blocks_test, add_test)
            if match:
                ft.write(" ".join(bc for bc in block_config) 
                        + "\t" + command + "\t" + " ".join(bc for bc in next_block_config) + "\n")
ft.close()

## Convert Dataset

In [2]:
import datetime
import numpy as np
import random
import itertools

In [3]:
def randomize_data(inps):
    remove_ind = np.arange(0, int(0.5 * len(inps)),1)
    add_ind = np.arange(int(0.5 * len(inps)) ,len(inps),1)
    random.shuffle(remove_ind)
    random.shuffle(add_ind)
    shuffled_index = np.array([val for pair in zip(remove_ind, add_ind) for val in pair])
    return np.array(inps)[shuffled_index].tolist()

In [4]:
def get_replaced_data(inps, color):
    res = []
    for a in inps:
        cnt = 0
        for col in color:
            if col in a:
                cnt += 1
        #if cnt == len(color):
        if cnt > 0:
            res.append(a)
    return res

In [5]:
color = ["remove"]
inps = []
with open("./dataset/lang_games_data_artificial_train_nvl_utter_blocks_50000.txt", "r") as f:
    for line in f:
        inps.append(line)
inps = randomize_data(inps)

In [26]:
f = open("./dataset/test_nvl_utter_blocks-remove.lisp", "w")
now = str(datetime.datetime.now())
for line in inps:
    target_blocks = []
    init_blocks = []
    ar = line.split("\t")
    i_blx = ar[0].replace("#","").replace(" ","").replace("\n","")
    t_blx = ar[2].replace("#","").replace(" ","").replace("\n","")
    start = 0
    for i in range(6):
        s_inner_block = []
        inner_block = []
        for j in range(3):
            if t_blx[start+j] != "X":
                inner_block.append(int(t_blx[start+j]))
            if i_blx[start+j] != "X":
                s_inner_block.append(int(i_blx[start+j]))
        start+=3
        target_blocks.append(inner_block)
        init_blocks.append(s_inner_block)
    strings = """(example\n  (id session:rezkatrial)
(context (date 2018 3 01) (graph NaiveKnowledgeGraph ((string """ + str(init_blocks).replace(" ","") + """) (name b) (name c))))
(timeStamp """ + now + """)\n  (NBestInd 0)\n  (utterance """ + '"' + ar[1] + '"' + """)\n  (targetFormula a)
(targetValue (string """ + str(target_blocks).replace(" ","") + """))\n)\n"""
    f.write(strings)
f.close()
#   (context (date 2016 1 29) (graph NaiveKnowledgeGraph ((string [[1,1],[1,3],[1,1],[1,3],[1,1],[1,3]]) (name b) (name c))))
#   (timeStamp 2016-02-29T12:41:54.832)
#   (NBestInd 0)
#   (utterance "remove orange")
#   (targetFormula
#     (call
#       edu.stanford.nlp.sempre.cubeworld.StacksWorld.wallToString
#       (call context:edu.stanford.nlp.sempre.cubeworld.StacksWorld.removeTop (call context:edu.stanford.nlp.sempre.cubeworld.StacksWorld.getTopColor (number 3 COLOR)))
#     )
#   )
#   (targetValue (string [[1,1],[1],[1,1],[1],[1,1],[1]]))
# )"

In [9]:
conf = [[["add", "red"], ["add", "orange"], ["add", "cyan"], ["add", "brown"]], ["test", "valid"]]
config = list(itertools.product(*conf))
print(config)
replace = {}
replace["remove"] = "rmv"
replace["add"] = "et"
replace["3rd"] = "3"
replace["every"] = "evr"
replace["brown"] = "braun"
replace["orange"] = "oranje"
replace["red"] = "roze"
replace["cyan"] = "cyaan"

[(['add', 'red'], 'test'), (['add', 'red'], 'valid'), (['add', 'orange'], 'test'), (['add', 'orange'], 'valid'), (['add', 'cyan'], 'test'), (['add', 'cyan'], 'valid'), (['add', 'brown'], 'test'), (['add', 'brown'], 'valid')]


In [7]:
list_files = glob.glob("./dataset/online_test/*.txt")
for line in list_files:
    start = './dataset/online_test/'
    end = ''
    fname = line[line.find(start)+len(start):line.rfind(end)]
    print(fname)

remove_brown_2nd.txt
cyan_4th.txt
remove_cyan.txt
cyan.txt
remove_cyan_6th.txt
remove_brown_4th.txt
remove_6th.txt
6th.txt
remove_brown_odd.txt
brown_every.txt
brown_6th.txt
brown.txt
remove_odd.txt
cyan_2nd.txt
all.txt
brown_4th.txt
remove_2nd.txt
remove_cyan_every.txt
remove_cyan_odd.txt
2nd.txt
remove_brown_every.txt
cyan_every.txt
every.txt
remove_brown.txt
cyan_6th.txt
remove_every.txt
cyan_odd.txt
4th.txt
brown_odd.txt
remove_cyan_4th.txt
remove_cyan_2nd.txt
remove_4th.txt
brown_2nd.txt
remove.txt
remove_brown_6th.txt


In [17]:
combine = False
for elem in list_files:
    inps = []
    with open(elem, "r") as f:
        for line in f:
            inps.append(line)
    if combine:
        inps_big = []
        with open("./dataset/lang_games_data_artificial_train_nvl_utter_blocks_50000.txt", "r") as f:
            for line in f:
                inps_big.append(line)
        fin_inps = inps_big + inps
    else:
        fin_inps = inps
    start = './dataset/online_test/'
    end = '.txt'
    fname = elem[elem.find(start)+len(start):elem.rfind(end)]
    f = open("./dataset/online_test/lisps/" + fname + "_sing.lisp", "w")
    now = str(datetime.datetime.now())
    for line in fin_inps:
        target_blocks = []
        init_blocks = []
        ar = line.split("\t")
        i_blx = ar[0].replace("#","").replace(" ","").replace("\n","")
        t_blx = ar[2].replace("#","").replace(" ","").replace("\n","")
#         for el in elem[0]:
#             uttr = ar[1].replace(el,replace[el])
        uttr = ar[1]
        start = 0
        for i in range(6):
            s_inner_block = []
            inner_block = []
            for j in range(3):
                if t_blx[start+j] != "X":
                    inner_block.append(int(t_blx[start+j]))
                if i_blx[start+j] != "X":
                    s_inner_block.append(int(i_blx[start+j]))
            start+=3
            target_blocks.append(inner_block)
            init_blocks.append(s_inner_block)
        strings = """(example\n  (id session:rezkatrial)
    (context (date 2018 3 01) (graph NaiveKnowledgeGraph ((string """ + str(init_blocks).replace(" ","") + """) (name b) (name c))))
    (timeStamp """ + now + """)\n  (NBestInd 0)\n  (utterance """ + '"' + uttr + '"' + """)\n  (targetFormula a)
    (targetValue (string """ + str(target_blocks).replace(" ","") + """))\n)\n"""
        f.write(strings)
    f.close()
    print(fname + "_sing")

remove_brown_2nd_sing
cyan_4th_sing
remove_cyan_sing
cyan_sing
remove_cyan_6th_sing
remove_brown_4th_sing
remove_6th_sing
6th_sing
remove_brown_odd_sing
brown_every_sing
brown_6th_sing
brown_sing
remove_odd_sing
cyan_2nd_sing
all_sing
brown_4th_sing
remove_2nd_sing
remove_cyan_every_sing
remove_cyan_odd_sing
2nd_sing
remove_brown_every_sing
cyan_every_sing
every_sing
remove_brown_sing
cyan_6th_sing
remove_every_sing
cyan_odd_sing
4th_sing
brown_odd_sing
remove_cyan_4th_sing
remove_cyan_2nd_sing
remove_4th_sing
brown_2nd_sing
remove_sing
remove_brown_6th_sing


In [6]:
vocab = ["remove", "add", "at", "tile", "1st", "2nd", "3rd", "4th", "5th", "6th", "leftmost", "rightmost", "red",
        "cyan", "orange", "brown", "every", "even", "odd"]
cor_files = []
with open("dataset/sida wang's/out.txt") as f:
    for line in f:
        cor_files.append(line.replace("\n",""))
for el in cor_files:
    overlap_count = 0
    try:
        with open("dataset/sida wang's/txt/" + el + ".txt") as ff:
            for line in ff:
                ar = line.split("\t")
                words = ar[1].split(" ")
                for word in words:
                    if word in vocab:
                        overlap_count += 1
            print(el, overlap_count)
    except:
        print(el, "not found")

0ECjYvl 48
1QPlUN8OJhM 2
2DdHNZKv5mM 99
81YlekTDMbQ 102
830kXNICuqb 0
8cEr4KwXjP9 2
8mtUgG0g0mz 60
8ZXjdVvTjga 70
A10G57EYF3HK46 0
A1123Z7CQMJ05R 0
A11NPNK8885N99 0
A1219SY7CR1UWP 0
A12VSA5C02M3FT 0
A13FF3UBS35JD7 0
A153FVJRRM27G7 2
A154X03NKVZZL1 0
A15781PHGW377Y 0
A15JP0TGZKLNIZ 0
A177EXELDLWTWV 0
A17AMTRIYI2Z2D 98
A17ELSQUMU0QT6 17
A1945USNZHTROX 30
A19ED8FYO6CA36 0
A19V9DLZS91SDL 65
A19XAU8SBQZLYM 49
A1A3TGZ7DKJWRW 99
A1BHHRSKO2MBT3 0
A1BI3WXOR5CTHQ 0
A1BWS5AD2T4NIR 12
A1C6BL1O5L5LA9 0
A1EAEZ7VIHKQ2 0
A1EITLFAMKA61U 13
A1F3TWJXGO0URG 0
A1F669OTXWIJW0 119
A1FBBY2JJYRMRI 93
A1FLNAFLLO3UPK 6
A1FP3SH704X01V 5
A1GQTXZPHAMLEH 4
A1GXFMAC759VRM 0
A1HKYY6XI2OHO1 1
A1IU5OP7BBZHZ7 3
A1IYLZCP01F9XT 0
A1JNC3HWWHJD2J 118
A1KK5PS8X2U5QP 120
A1M682B2WUSYJP 84
A1MIR0TP081SKT 0
A1MYLQQL8BBOYT 6
A1N84E64VEYY22 0
A1NFBKUYYVUYWF 4
A1NKBXOTZAI1YK 13
A1OK974KXKO6FJ 109
A1P6OXEJ86HQRM 0
A1SWLR0UPOFRCW 169
A1T02KY1E9YI3I 2
A1T9IKE8GV4LR4 0
A1TH0PTGDSBWMO 5
A1TL8OAM2V1Q5K 1
A1U5BE8XJRXKW3 116
A1VI3SOIHT6Y0D